In [2]:
# 1단계 - 헤더 검증 및 연속적인 요청 패턴 회피
import requests
import time
import random

# 4가지 헤더 추가해야함
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36',
    'Referer': 'http://127.0.0.1:5000/level3',
    'Accept': 'application/json',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

for i in range(1, 20):
    params = {
        "page": i,
        "pageSize": 30
    }

    response = requests.get("http://127.0.0.1:5000/protected_api/articles", params=params, headers=headers)

    # 페이지 번호가 유효하지 않은 경우 종료
    if response.status_code == 400:
        break

    # 페이지 번호가 유효한 경우 게시글 번호 출력
    articles = response.json()
    for article in articles:
        id = article['article_no']
        print(f'매물번호 : {id}')
    
    # 연속적인 요청 시 차단되는 것 방지
    time.sleep(random.uniform(1, 1.5))

매물번호 : 2456802095
매물번호 : 2456645976
매물번호 : 2460477200
매물번호 : 2461781941
매물번호 : 2460813114
매물번호 : 2460519930
매물번호 : 2460211781
매물번호 : 2461115789
매물번호 : 2459776757
매물번호 : 2461933668
매물번호 : 2461968304
매물번호 : 2462143478
매물번호 : 2462079959
매물번호 : 2461710895
매물번호 : 2456968913
매물번호 : 2462092179
매물번호 : 2461677544
매물번호 : 2461406767
매물번호 : 2461414115
매물번호 : 2461969152
매물번호 : 2458607738
매물번호 : 2459312425
매물번호 : 2458650523
매물번호 : 2458912020
매물번호 : 2461707779
매물번호 : 2460837593
매물번호 : 2459588566
매물번호 : 2461493495
매물번호 : 2460930247
매물번호 : 2457223454
매물번호 : 2460572058
매물번호 : 2460571572
매물번호 : 2461512651
매물번호 : 2459494327
매물번호 : 2458717384
매물번호 : 2458525959
매물번호 : 2458108306
매물번호 : 2462115896
매물번호 : 2456721584
매물번호 : 2457282343
매물번호 : 2461582653
매물번호 : 2461581710
매물번호 : 2460791515
매물번호 : 2460732509
매물번호 : 2461807188
매물번호 : 2461528632
매물번호 : 2461526515
매물번호 : 2460362847
매물번호 : 2461545576
매물번호 : 2460538270
매물번호 : 2458890542
매물번호 : 2458264193
매물번호 : 2462057766
매물번호 : 2461926032
매물번호 : 2460487927
매물번호 : 245

In [4]:
# 2단계 - 세션 검증 회피
import requests
import time
import random
import re, json
from selenium import webdriver

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36',
    'Referer': 'http://127.0.0.1:5000/level3',
    'Accept': 'application/json',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',    
}

def new_session():
    # 브라우저 열기
    driver = webdriver.Chrome()
    driver.get("http://127.0.0.1:5000/level3")
    time.sleep(random.uniform(1, 1.5))

    # 쿠키 가져오기
    cookies = driver.get_cookies()
    driver.quit()

    # 세션 생성
    session = requests.Session()

    # 쿠키 설정
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])

    return session


if __name__ == "__main__":
    session = new_session()
    session.headers.update(HEADERS)

    for i in range(1, 20):
        params = {
            "page": i,
            "pageSize": 30
        }

        response = session.get("http://127.0.0.1:5000/protected_api/articles", params=params)
        print(f'{i}페이지요청 : {response.status_code}')

        if response.status_code == 400:
            break
        articles = response.json()
        for article in articles:
            id = article['article_no']

            while True:
                response = session.get(f"http://127.0.0.1:5000/level3/{id}")
                print(f'매물 {id}요청 : {response.status_code}')
                
                if response.status_code == 401:
                    session = new_session()
                    session.headers.update(HEADERS)
                else:
                    pattern = r'window\.__PRELOADED_STATE__\s*=\s*(\{.*?\});'
                    match = re.search(pattern, response.text, re.DOTALL)

                    if match:
                        json_text = match.group(1)
                        # 딕셔너리로 변환
                        product_data = json.loads(json_text)
                        article_realtor_name = product_data.get('article_realtor_name', '')
                        article_realtor_address = product_data.get('article_realtor_address', '')
                        print(f"부동산명 : {article_realtor_name}") 
                        print(f"부동산주소 : {article_realtor_address}\n")
                    else:
                        print("JSON을 찾을 수 없습니다.")
                    break
            time.sleep(random.uniform(1, 1.5))

        time.sleep(random.uniform(1, 1.5))

1페이지요청 : 200
매물 2456802095요청 : 200
부동산명 : 더퍼플공인중개사사무소
부동산주소 : 서울특별시 강남구 대치동 889-41

매물 2456645976요청 : 200
부동산명 : 뉴황금공인중개사사무소
부동산주소 : 서울특별시 강남구 역삼로63길 6 2층

매물 2460477200요청 : 200
부동산명 : 빌딩나비강남역쉐르빌부동산중개주식회사
부동산주소 : 서울특별시 강남구 강남대로 328 1층 110호 (역삼동)

매물 2461781941요청 : 200
부동산명 : 강남플러스부동산중개법인
부동산주소 : 서울 강남구 역삼동827-14 1층 102-2호(역삼동, 프레스티지투)

매물 2460813114요청 : 200
부동산명 : 주식회사 우대빵부동산중개센터 강남지점
부동산주소 : 서울 강남구 언주로 425,2층

매물 2460519930요청 : 200
부동산명 : 빌딩나비강남역쉐르빌부동산중개주식회사
부동산주소 : 서울특별시 강남구 강남대로 328 1층 110호 (역삼동)

매물 2460211781요청 : 200
부동산명 : 부자부동산공인중개사사무소
부동산주소 : 서울 강남구 대치동1030 1층 S109호 (대치동 상가)

매물 2461115789요청 : 200
부동산명 : 에스디(SD)공인중개사 사무소
부동산주소 : 서울특별시 서초구 잠원동 34-14 1층 01호

매물 2459776757요청 : 200
부동산명 : 신한공인중개사사무소
부동산주소 : 서울특별시 강남구 선릉로 571,지하2층 1호(역삼동)

매물 2461933668요청 : 200
부동산명 : 강남최고공인중개사사무소
부동산주소 : 서울 강남구 언주로86길 11,305호(역삼동, 한화진넥스빌)

매물 2461968304요청 : 200
부동산명 : 주식회사 우대빵부동산중개센터 강남지점
부동산주소 : 서울 강남구 언주로 425,2층

매물 2462143478요청 : 200
부동산명 : 오렌지공인중개사사무소
부동산주소 : 서울특별시 강남구 역삼동 754-2 도곡프라자 110호 오렌지부

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# 3단계 - 요청횟수 제한 및 기타 서버 오류 회피
import requests
import time
import random
from selenium import webdriver

def new_session():
    """브라우저 세션 생성"""
    driver = webdriver.Chrome()
    driver.get("http://127.0.0.1:5000/level3")
    time.sleep(random.uniform(1, 1.5))

    cookies = driver.get_cookies()
    driver.quit()

    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])
        
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36',
        'Referer': 'http://127.0.0.1:5000/level3',
        'Accept': 'application/json',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    })

    return session

def make_request(session, url, params=None, max_retries=3):
    """재시도 로직이 포함된 요청 함수"""
    for attempt in range(max_retries):
        try:
            response = session.get(url, params=params)
            
            # 성공
            if response.status_code == 200:
                return response, "success"
                
            # 세션 만료
            elif response.status_code == 401:
                print("세션이 만료되었습니다.")
                return None, "expired"
                
            # IP 차단
            elif response.status_code == 403:
                print(f"IP가 차단되었습니다. 재시도 {attempt + 1}/{max_retries}")
                
            # 기타 오류
            else:
                print(f"요청 실패: {response.status_code} (재시도 {attempt + 1}/{max_retries})")

            wait_time = (attempt + 1) * 10
            time.sleep(wait_time)
            continue
            
        except requests.exceptions.RequestException as e:
            print(f"요청 오류 발생: {e} (재시도 {attempt + 1}/{max_retries})")
            wait_time = (attempt + 1) * 10
            time.sleep(wait_time)
            continue
    
    return None, "failed"

def main():
    session = new_session()
    
    for page in range(1, 20):
        while True:  # 페이지 요청 성공할 때까지 반복
            response, status = make_request(
                session, 
                "http://127.0.0.1:5000/protected_api/articles",
                params={"page": page, "pageSize": 30}
            )
            
            # 세션 만료시 즉시 새로운 세션 생성
            if status == "expired":
                print("새로운 세션을 생성합니다...")
                session = new_session()
                continue
                
            # 성공한 경우
            if status == "success":
                print(f'{page}페이지 요청 성공: {response.status_code}')
                try:
                    articles = response.json()
                    break  # 성공적으로 데이터를 받았으면 반복 종료
                except ValueError:
                    print("JSON 파싱 오류")
                    break
            
        # 각 매물 상세 정보 요청
        for article in articles:
            article_id = article['article_no']
            
            while True:  # 매물 요청 성공할 때까지 반복
                response, status = make_request(
                    session,
                    f"http://127.0.0.1:5000/level3/{article_id}"
                )
                
                # 세션 만료시 즉시 새로운 세션 생성
                if status == "expired":
                    print("새로운 세션을 생성합니다...")
                    session = new_session()
                    continue
                    
                # 성공한 경우
                if status == "success":
                    print(f'매물 {article_id} 요청 성공: {response.status_code}')
                    break
            
            # 요청 간격 랜덤화
            time.sleep(random.uniform(1, 1.5))
            
        # 페이지 간격 랜덤화
        time.sleep(random.uniform(1, 1.5))

if __name__ == "__main__":
    main()

1페이지 요청 성공: 200
매물 2456802095 요청 성공: 200
매물 2456645976 요청 성공: 200
매물 2460477200 요청 성공: 200
매물 2461781941 요청 성공: 200
매물 2460813114 요청 성공: 200
매물 2460519930 요청 성공: 200
매물 2460211781 요청 성공: 200
매물 2461115789 요청 성공: 200
매물 2459776757 요청 성공: 200
매물 2461933668 요청 성공: 200
매물 2461968304 요청 성공: 200
매물 2462143478 요청 성공: 200
매물 2462079959 요청 성공: 200
매물 2461710895 요청 성공: 200
매물 2456968913 요청 성공: 200
매물 2462092179 요청 성공: 200
매물 2461677544 요청 성공: 200
매물 2461406767 요청 성공: 200
매물 2461414115 요청 성공: 200
매물 2461969152 요청 성공: 200
매물 2458607738 요청 성공: 200
매물 2459312425 요청 성공: 200
매물 2458650523 요청 성공: 200
매물 2458912020 요청 성공: 200
매물 2461707779 요청 성공: 200
매물 2460837593 요청 성공: 200
매물 2459588566 요청 성공: 200
매물 2461493495 요청 성공: 200
매물 2460930247 요청 성공: 200
매물 2457223454 요청 성공: 200
2페이지 요청 성공: 200
매물 2460572058 요청 성공: 200
매물 2460571572 요청 성공: 200
매물 2461512651 요청 성공: 200
매물 2459494327 요청 성공: 200
매물 2458717384 요청 성공: 200
매물 2458525959 요청 성공: 200
매물 2458108306 요청 성공: 200
매물 2462115896 요청 성공: 200
매물 2456721584 요청 성

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


매물 2461807188 요청 성공: 200
매물 2461528632 요청 성공: 200
매물 2461526515 요청 성공: 200
매물 2460362847 요청 성공: 200
매물 2461545576 요청 성공: 200
매물 2460538270 요청 성공: 200
매물 2458890542 요청 성공: 200
매물 2458264193 요청 성공: 200
매물 2462057766 요청 성공: 200
매물 2461926032 요청 성공: 200
매물 2460487927 요청 성공: 200
매물 2459973276 요청 성공: 200
매물 2460813127 요청 성공: 200
매물 2460133369 요청 성공: 200
매물 2461160649 요청 성공: 200
매물 2456443608 요청 성공: 200
매물 2459291465 요청 성공: 200
매물 2457371866 요청 성공: 200
매물 2458270446 요청 성공: 200
매물 2458437750 요청 성공: 200
13페이지 요청 성공: 200
매물 2461971849 요청 성공: 200
매물 2461993435 요청 성공: 200
매물 2461768359 요청 성공: 200
매물 2461080453 요청 성공: 200
매물 2459353031 요청 성공: 200
매물 2460724278 요청 성공: 200
매물 2460904833 요청 성공: 200
매물 2458574458 요청 성공: 200
매물 2458437712 요청 성공: 200
매물 2461274629 요청 성공: 200
매물 2460477072 요청 성공: 200
매물 2460247152 요청 성공: 200
매물 2459696763 요청 성공: 200
매물 2458703205 요청 성공: 200
매물 2461784965 요청 성공: 200
매물 2460496702 요청 성공: 200
매물 2459492668 요청 성공: 200
매물 2457823356 요청 성공: 200
매물 2457350368 요청 성공: 200
매물 24620